In [1]:
import sys
sys.path.append("..")

import torch
import pickle
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision import transforms as T
from PIL import Image
import numpy as np
import json
import os
print(os.getcwd())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
datapath = os.getcwd()
datasetTrain = 0.7
datasetVal = 0.15
datasetSize = 10336

/home/gijsf/Documents/Thesis/2D/data


In [2]:
#Load the SunRGB dataset in the coco format
NYU40CLASSES = ['void',
                'wall', 'floor', 'cabinet', 'bed', 'chair',
                'sofa', 'table', 'door', 'window', 'bookshelf',
                'picture', 'counter', 'blinds', 'desk', 'shelves',
                'curtain', 'dresser', 'pillow', 'mirror', 'floor_mat',
                'clothes', 'ceiling', 'books', 'refridgerator', 'television',
                'paper', 'towel', 'shower_curtain', 'box', 'whiteboard',
                'person', 'night_stand', 'toilet', 'sink', 'lamp',
                'bathtub', 'bag', 'otherstructure', 'otherfurniture', 'otherprop'] 

def arrayFromSUNRGBD_to_coco(pathToData, startIdDataset , stopIdDataset):
    coco_dict = {
        'images': [],
        'annotations': [],
        'categories': []
    }
    category_set = set()
    annotation_id = 0
    for i in range(startIdDataset, stopIdDataset):
        try:
            with (open(pathToData + "/pickles/" + str(i) + ".pkl", "rb")) as openfile:
                dataFromPickle = pickle.load(openfile)
                
            #Check if the image has no labels
            if len(dataFromPickle["boxes"]["size_cls"]) == 0:
                continue
            
            #check if all labels are VOID  
            if all(label == 0 for label in dataFromPickle["boxes"]["size_cls"]):
                continue
            
            image_id = i - startIdDataset
            image_info = {
                'id': image_id,
                'file_name': pathToData + "/photos/time_data/" + str(i) + "/img.jpg",
                # You may need to add 'width' and 'height' fields
            }
            coco_dict['images'].append(image_info)

            boxes = dataFromPickle["boxes"]["bdb2D_pos"]
            labels = dataFromPickle["boxes"]["size_cls"]
            for box, label in zip(boxes, labels):
                if label == 0:
                    continue
                
                category_set.add(label)
                annotation_info = {
                    'id': annotation_id,
                    'image_id': image_id,
                    'category_id': label,  # You may need to map label to category_id
                    #data was in xmin, ymin, xmax, ymax format convert to x, y, width, height
                    'bbox': [box[0], box[1], int(box[2] - box[0]), int(box[3] - box[1])], 
                    'area': float(np.abs(box[2] - box[0] * box[3] - box[1])),
                    'iscrowd': 0  # Assuming no crowd
                }
                coco_dict['annotations'].append(annotation_info)
                annotation_id += 1

        except:
            pass

    for category_id, category_name in enumerate(category_set):
        category_info = {
            'id': category_id,
            'name': category_name
        }
        coco_dict['categories'].append(category_info)

    return coco_dict

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)
        
# Convert the dataset to a dictionary in the COCO format
coco_dict = arrayFromSUNRGBD_to_coco(datapath,1,int(datasetSize*datasetTrain))
with open('coco_dataset_train.json', 'w') as f:
    json.dump(coco_dict, f, cls=NumpyEncoder)
    print("Size of the training JSON: ", f.tell())

coco_dict = arrayFromSUNRGBD_to_coco(datapath,int(datasetSize*datasetTrain),int((datasetTrain+datasetVal)*datasetSize))
with open('coco_dataset_val.json', 'w') as f:
    json.dump(coco_dict, f, cls=NumpyEncoder)
    print("Size of the validation JSON: ", f.tell())

coco_dict = arrayFromSUNRGBD_to_coco(datapath,int((datasetTrain+datasetVal)*datasetSize),datasetSize)
with open('coco_dataset_test.json', 'w') as f:
    json.dump(coco_dict, f, cls=NumpyEncoder)
    print("Size of the testing JSON: ", f.tell())

Size of the training JSON:  4234842
Size of the validation JSON:  773435
Size of the testing JSON:  663955
